In [1]:
import numpy as np
from math import log
import time

In [2]:
start = time.time()
num=10000#训练集样本数
d1=1000#测试集样本数
d2=0#测试集样本中判断正确个数的初值
d3=20#（0,1）间分为几份
X=[[0.3,0.7],[0.7,0.3]]#两正态分布的中心

In [3]:
#坐标转换
def cf(i):
    c=np.floor(i*d3)+1
    #这里x1/x2都分为20类，否则需分类定义
    return c

In [4]:
#生成训练集,第一列为类别，第二列为第一个特征，第三列为第二个特征
aa=np.empty((num,3))
for i in range(0,int(0.6*num)):
    aa[i][0]=0
    aa[i][1]=cf(np.random.normal(0.3,np.sqrt(0.4)))
    aa[i][2]=cf(np.random.normal(0.7,np.sqrt(0.4)))

for i in range(int(0.6*num),num):
    aa[i][0]=1
    aa[i][1]=cf(np.random.normal(0.7,np.sqrt(0.4)))
    aa[i][2]=cf(np.random.normal(0.3,np.sqrt(0.4)))

In [5]:
#得到了压缩后的训练集，包含了训练决策树算法需要的所有信息
#截取0到1的压缩训练集数据，并将数据到这400个格子中
#bb[j][k][0]表示第j行第k列中类0的数目，bb[j][k][1]表示第j行第k列中类1的数目
bb=np.zeros((d3,d3,2))
#（0,1）内总样本数c1
c1=0
for i in range(num):
    if aa[i][1]>0 and aa[i][1]<=d3 and aa[i][2]>0 and aa[i][2]<=d3 :
        c1+=1
        j=int(aa[i][1])-1
        k=int(aa[i][2])-1
        if aa[i][0]==0:
            bb[j][k][0]+=1
        else:
            bb[j][k][1]+=1

In [6]:
cc=np.zeros((20,2))
for i in range(20):
    cc[i][0]=bb[:,i,0].sum()
    cc[i][1]=bb[:,i,1].sum()

dd=np.zeros((20,2))
for i in range(20):
    dd[i][0]=bb[i,:,0].sum()
    dd[i][1]=bb[i,:,1].sum()


In [7]:
#根节点的熵为0.684
def prop(x,y):
    #定义一个计算x中y类样本说占比例
    d=(x[:,:,y].sum())/c1
    #d=(sum(sum(x[:,:,y])))/c1
    return d

bbentropy=-(prop(bb,1)*log(prop(bb,1))+prop(bb,0)*log(prop(bb,0)))
bbentropy

0.6624024241288493

In [8]:
#按行或列(y=0或1)划分后,第z类样本中第z类(20类)所占比例Di/D
def prop1(y,z):
    if y==0:
        return cc[z].sum()/c1
    if y==1:
        return dd[z].sum()/c1
#H(Di),y为类别，z为i
def entropy1(y,z):
    if y==0:
        return -((cc[z][0]/cc[z].sum())*log(cc[z][0]/cc[z].sum())+(cc[z][1]/cc[z].sum())*log(cc[z][1]/cc[z].sum()))
    else:
        return -((dd[z][0]/dd[z].sum())*log(dd[z][0]/dd[z].sum())+(dd[z][1]/dd[z].sum())*log(dd[z][1]/dd[z].sum()))

In [9]:
#定义条件熵，类别y=0或1
def entropy2(y):
    b=0
    for z in range(20):
        b+=prop1(y,z)*entropy1(y,z)
    return b


In [10]:
#FALSE代表用横坐标划分后的条件熵比用纵坐标划分后的条件熵大，先用纵坐标划分信息增益大
#True代表用横坐标划分后的条件熵比用纵坐标划分后的条件熵小，先用横坐标划分信息增益大
entropy2(1)>entropy2(0)

False

In [11]:
def I(c):
    n=np.random.uniform()
    if c<=n:
        return 0
    else:
        return 1
#生成测试集
ee=np.empty((d1,3),int)
for i in range(d1):
    #cc[i][0]=i+1#cc[i]的序号
    ee[i][0]=I(0.4)#cc[i]的真实类
    a=int(ee[i][0])
    ee[i][1]=cf(np.random.normal(X[a][0],np.sqrt(0.4)))
    ee[i][2]=cf(np.random.normal(X[a][1],np.sqrt(0.4)))

In [12]:
#裁剪测试集
ff=[]
for i in range(d1):
    if ee[i][1]>0 and ee[i][1]<=d3 and ee[i][2]>0 and ee[i][2]<=d3:
        ff.append(ee[i])
ff=np.array(ff,int)

In [13]:
#决策树的叶结点
gg=np.empty((d3,d3))
k=0
for i in range(d3):
    for j in range(d3): 
        if bb[i][j][0]>=bb[i][j][1]:
            gg[i][j]=0
            k+=1
        else:
            gg[i][j]=1

In [14]:
su=0
for i in range(len(ff)):
    if ff[i][0]==gg[ff[i][1]-1,ff[i][2]-1]:
        su+=1
su/len(ff)


0.6236933797909407